In [1]:
from fishsense_data_processing_workflow_worker.database import Database
from sqlmodel.ext.asyncio.session import AsyncSession
import asyncio
from pathlib import Path
from skimage.util import img_as_ubyte
from fishsense.image.image_processors.raw_processor import RawProcessor
from fishsense.calibration import LensCalibration, LaserCalibration
import matplotlib.pyplot as plt
from fishsense.image import ImageRectifier
import cv2
import numpy as np
from label_studio_sdk.client import LabelStudio

In [3]:
database = Database(PG_CONNECTION_STRING)

In [4]:
clusters = await database.select_dive_frame_clusters()

clusters

[DiveFrameCluster(dive_id=279, id=621),
 DiveFrameCluster(dive_id=279, id=622),
 DiveFrameCluster(dive_id=279, id=623),
 DiveFrameCluster(dive_id=279, id=624),
 DiveFrameCluster(dive_id=279, id=625),
 DiveFrameCluster(dive_id=279, id=626),
 DiveFrameCluster(dive_id=279, id=627),
 DiveFrameCluster(dive_id=279, id=628),
 DiveFrameCluster(dive_id=279, id=629),
 DiveFrameCluster(dive_id=279, id=630),
 DiveFrameCluster(dive_id=279, id=631),
 DiveFrameCluster(dive_id=279, id=632),
 DiveFrameCluster(dive_id=279, id=633),
 DiveFrameCluster(dive_id=279, id=634),
 DiveFrameCluster(dive_id=279, id=635),
 DiveFrameCluster(dive_id=279, id=636),
 DiveFrameCluster(dive_id=279, id=637),
 DiveFrameCluster(dive_id=279, id=638),
 DiveFrameCluster(dive_id=279, id=639),
 DiveFrameCluster(dive_id=279, id=640),
 DiveFrameCluster(dive_id=279, id=641),
 DiveFrameCluster(dive_id=279, id=642),
 DiveFrameCluster(dive_id=279, id=643),
 DiveFrameCluster(dive_id=279, id=644),
 DiveFrameCluster(dive_id=279, id=645),


In [5]:
async with AsyncSession(database.engine) as session:
    cluster_mappings = await asyncio.gather(*(database.select_dive_frame_cluster_image_mappings_by_cluster_id(cluster.id, session=session) for cluster in clusters))

cluster_mappings

[[DiveFrameClusterImageMapping(image_id=78666, dive_frame_cluster_id=621),
  DiveFrameClusterImageMapping(image_id=78667, dive_frame_cluster_id=621)],
 [DiveFrameClusterImageMapping(image_id=78670, dive_frame_cluster_id=622),
  DiveFrameClusterImageMapping(image_id=78669, dive_frame_cluster_id=622),
  DiveFrameClusterImageMapping(image_id=78671, dive_frame_cluster_id=622),
  DiveFrameClusterImageMapping(image_id=78668, dive_frame_cluster_id=622)],
 [DiveFrameClusterImageMapping(image_id=78674, dive_frame_cluster_id=623),
  DiveFrameClusterImageMapping(image_id=78673, dive_frame_cluster_id=623),
  DiveFrameClusterImageMapping(image_id=78672, dive_frame_cluster_id=623)],
 [DiveFrameClusterImageMapping(image_id=78675, dive_frame_cluster_id=624),
  DiveFrameClusterImageMapping(image_id=78676, dive_frame_cluster_id=624),
  DiveFrameClusterImageMapping(image_id=78677, dive_frame_cluster_id=624)],
 [DiveFrameClusterImageMapping(image_id=78678, dive_frame_cluster_id=625),
  DiveFrameClusterIma

In [6]:
async with AsyncSession(database.engine) as session:
    images = await asyncio.gather(*(database.select_images_by_ids([mapping.image_id for mapping in mappings], session=session) for mappings in cluster_mappings))

images

[[Image(path='drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05/PB130168.ORF', taken_datetime=datetime.datetime(2023, 11, 13, 12, 3, 31, tzinfo=datetime.timezone.utc), is_canonical=True, camera_id=5, checksum='7252dfb0db0b855616e45607f58c1c93', id=78666, dive_id=279),
  Image(path='drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05/PB130169.ORF', taken_datetime=datetime.datetime(2023, 11, 13, 12, 3, 31, tzinfo=datetime.timezone.utc), is_canonical=True, camera_id=5, checksum='89832e9de79aac7aacdc7091540ff36a', id=78667, dive_id=279)],
 [Image(path='drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05/PB130173.ORF', taken_datetime=datetime.datetime(2023, 11, 13, 12, 3, 32, tzinfo=datetime.timezone.utc), is_canonical=True, camera_id=5, checksum='b7248d8c1b3c845ca3404cc

In [7]:
image_subset = images[34]

len(image_subset), image_subset

(4,
 [Image(path='drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05/PB130270.ORF', taken_datetime=datetime.datetime(2023, 11, 13, 12, 24, 15, tzinfo=datetime.timezone.utc), is_canonical=True, camera_id=5, checksum='965e90b85f5cd8bc0860bb02176fe01c', id=78765, dive_id=279),
  Image(path='drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05/PB130267.ORF', taken_datetime=datetime.datetime(2023, 11, 13, 12, 24, 15, tzinfo=datetime.timezone.utc), is_canonical=True, camera_id=5, checksum='d24b71a96ba916ebf54dc60220336b25', id=78766, dive_id=279),
  Image(path='drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05/PB130269.ORF', taken_datetime=datetime.datetime(2023, 11, 13, 12, 24, 15, tzinfo=datetime.timezone.utc), is_canonical=True, camera_id=5, checksum='613bd0c40cfeed67e

In [8]:
data_path = Path("./data/REEF/data").absolute()

data_path

PosixPath('/home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/scripts/data/REEF/data')

In [9]:
image_paths = [data_path / image.path for image in image_subset]

image_paths, all(path.exists() for path in image_paths)

([PosixPath('/home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/scripts/data/REEF/data/drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05/PB130270.ORF'),
  PosixPath('/home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/scripts/data/REEF/data/drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05/PB130267.ORF'),
  PosixPath('/home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/scripts/data/REEF/data/drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05/PB130269.ORF'),
  PosixPath('/home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/scripts/data/REEF/data/drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator De

In [10]:
await database.select_camera_by_id(image_subset[0].camera_id)

Camera(serial_number='BJ6C67988', name='FSL-05', id=5)

In [11]:
calibration_path = Path("./calib/FishSense Lite Calibration Parameters/FSL-05D/fsl-05d-lens-raw.pkg").absolute()

calibration_path.exists(), calibration_path

(True,
 PosixPath('/home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/scripts/calib/FishSense Lite Calibration Parameters/FSL-05D/fsl-05d-lens-raw.pkg'))

In [12]:
target_path = Path("./process_work/preprocess_groups_jpeg").absolute()

target_path.exists(), target_path

(True,
 PosixPath('/home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/scripts/process_work/preprocess_groups_jpeg'))

In [13]:
lens_calibration = LensCalibration()
lens_calibration.load(calibration_path)

lens_calibration

In [14]:
tasks = []

raw_processor = RawProcessor()
image_rectifier = ImageRectifier(lens_calibration)

for idx, (image, image_path) in enumerate(zip(image_subset, image_paths)):
    img = raw_processor.process(image_path)
    img = img_as_ubyte(image_rectifier.rectify(img))

    height, width = img.shape[:2]

    # Define text properties
    text = f"{idx + 1}/{len(image_paths)}"
    org = (width - 350, height - 75) # Bottom-right corner of the text
    fontFace = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 5
    color = (0, 0, 255) # Red color (BGR)
    thickness = 10
    lineType = cv2.LINE_AA

    # Put the text on the image
    cv2.putText(img, text, org, fontFace, fontScale, color, thickness, lineType)

    cv2.imwrite(target_path / f"{image.checksum}.JPG", img)

    laser_label = await database.select_laser_label_by_image_id(image.id)
    if laser_label is None:
        predictions = []
    else:
        predictions = [{
            "model_version": "sql",
            "result": [{
                "id": "result1",
                "type": "keypointlabels",        
                "to_name":"image","from_name":"laser",
                "original_width": width, "original_height": height,
                "image_rotation": 0,
                "value": {
                    "x": laser_label.x / width * 100,
                    "y": laser_label.y / height * 100,
                    "width": 0.2,
                    "keypointlabels": [laser_label.label]
                }
            }]
        }]

    tasks.append({
        "data": {
            "image": f"https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/groups_jpeg/{image.checksum}"
        },
        "predictions": predictions,
        "annotations": []
    })

In [ ]:
client = LabelStudio(base_url=f"https://labeler.e4e.ucsd.edu", api_key="")

In [16]:
client.projects.import_tasks(57, request=tasks)

tasks

[{'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/groups_jpeg/965e90b85f5cd8bc0860bb02176fe01c'},
  'predictions': [{'model_version': 'sql',
    'result': [{'id': 'result1',
      'type': 'keypointlabels',
      'to_name': 'image',
      'from_name': 'laser',
      'original_width': 4014,
      'original_height': 3016,
      'image_rotation': 0,
      'value': {'x': 48.59269188288886,
       'y': 41.74462739315546,
       'width': 0.2,
       'keypointlabels': ['Red Laser']}}]}],
  'annotations': []},
 {'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/groups_jpeg/d24b71a96ba916ebf54dc60220336b25'},
  'predictions': [{'model_version': 'sql',
    'result': [{'id': 'result1',
      'type': 'keypointlabels',
      'to_name': 'image',
      'from_name': 'laser',
      'original_width': 4014,
      'original_height': 3016,
      'image_rotation': 0,
      'value': {'x': 48.69049878208928,
       'y': 42.523840529409654,
       'width': 0.2